In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import cv2
import os
from datetime import timedelta
import sys
from pathlib import Path 

In [2]:
def format_timedelta(td):
    """Служебная функция для классного форматирования объектов timedelta (например, 00:00:20.05)
    исключая микросекунды и сохраняя миллисекунды"""
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return (result + ".00").replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":", "-")




In [ ]:
def main(video_file):
    """
    Функция возвращает список тайм-кодов с нарушениями в виде списка из кортежей,
    кортеж: (время начала нарушения, время окончания нарушения)
    (формат времени: чч:мм:сс.мс)
    
    Args:
        video_file - путь до видео

    """
    violation_time_codes = [] #список тайм-кодов нарушений
    
    # читать видео файл    
    cap = cv2.VideoCapture(video_file)
    # получить FPS видео
    fps = cap.get(cv2.CAP_PROP_FPS)
    # если SAVING_FRAMES_PER_SECOND выше видео FPS, то установите его на FPS (как максимум)
    saving_frames_per_second = min(fps, SAVING_FRAMES_PER_SECOND)
    # получить список длительностей для сохранения
    
    # получаем продолжительность клипа, разделив количество кадров на количество кадров в секунду
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
    
    # список тпйм-кодов для стоп-кадров    
    list_time_codes = []    
    for i in np.arange(0, clip_duration, 1 / saving_frames_per_second):
        list_time_codes.append(i)
     
    # запускаем цикл
    count = 0
    flag_old = 0 # предыдущий флаг
    while True:
        is_read, frame = cap.read()
        if not is_read:
            # выйти из цикла, если нет фреймов для чтения
            break
        # получаем продолжительность, разделив количество кадров на FPS
        frame_duration = count / fps
        try:
            # получить самую раннюю продолжительность для сохранения
            closest_duration = list_time_codes[0]
        except IndexError:
            # список пуст, все кадры длительности сохранены
            break
        if frame_duration >= closest_duration:
            # если ближайшая длительность меньше или равна длительности кадра,
            # затем сохраняем фрейм
            frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))           
            
            #  ВЫЗОВ ИНФЕРЕНСА 
            # =================================================================
#             flag_new = inference()
            #==================================================================
            # обработка полученного флага
            if flag_new == 1 and flag_old == 0:
                start_violation = frame_duration_formatted
                
            if flag_new == 0 and flag_old == 1:               
                end_violation = frame_duration_formatted
                violation_time_codes.append((start_violation, end_violation))
                
            flag_old = flag_new
            # удалить точку продолжительности из списка, так как эта точка длительности уже сохранена
            try:
                list_time_codes.pop(0)
            except IndexError:
                pass
        # увеличить количество кадров
        count += 1
    return violation_time_codes